In [1]:
import pandas as pd
import numpy as np

In [2]:
pokemon=pd.read_csv('./data/pokemons_data.csv',index_col=0)
combats=pd.read_csv('./data/combats.csv')
pokemon.head(5)

,Name,Hit Points,Attack,Defense,Sp. Atk,Sp. Def,Speed,Legendary
#,,,,,,,,
1,Bulbasaur,45,49,49,65,65,45,0
2,Ivysaur,60,62,63,80,80,60,0
3,Venusaur,80,82,83,100,100,80,0
4,Mega Venusaur,80,100,123,122,120,80,0
5,Charmander,39,52,43,60,50,65,0


In [3]:
def normalization(data_df):
    stats=["Hit Points","Attack","Defense","Sp. Atk","Sp. Def","Speed","Legendary"]
    stats_df=pokemon[stats].T.to_dict("list")
    one=data_df.First_pokemon.map(stats_df)
    two=data_df.Second_pokemon.map(stats_df)
    temp_list=[]
    for i in range(len(one)):
        temp_list.append(np.array(one[i])-np.array(two[i]))
    new_test = pd.DataFrame(temp_list, columns=stats)
    for c in stats:
        description=new_test[c].describe()
        new_test[c]=(new_test[c]-description['min'])/(description['max']-description['min'])
    return new_test

In [4]:
combats.head(5)

,First_pokemon,Second_pokemon,Winner
0,266,298,298
1,702,701,701
2,191,668,668
3,237,683,683
4,151,231,151


In [6]:
cols = ["First_pokemon","Second_pokemon","Winner"]
new_combat_data=combats[cols].replace(pokemon.Name)

In [5]:
combats.Winner[combats.Winner == combats.First_pokemon] = 0
combats.Winner[combats.Winner == combats.Second_pokemon] = 1
combats.head(5)

,First_pokemon,Second_pokemon,Winner
0,266,298,1
1,702,701,1
2,191,668,1
3,237,683,1
4,151,231,0


In [6]:
data=normalization(combats)
data = pd.concat([data,combats.Winner], axis=1)
data.head(5)

,Hit Points,Attack,Defense,Sp. Atk,Sp. Def,Speed,Legendary,Winner
0,0.455556,0.468571,0.517241,0.450425,0.527578,0.450746,0.5,1
1,0.500000,0.374286,0.452874,0.543909,0.597122,0.507463,0.5,1
2,0.455556,0.385714,0.517241,0.365439,0.527578,0.507463,0.5,1
3,0.417778,0.257143,0.379310,0.521246,0.383693,0.423881,0.5,1
4,0.611111,0.628571,0.252874,0.790368,0.119904,0.656716,0.5,0


In [7]:
x_label=data.drop("Winner",axis=1)
y_label=data["Winner"]

In [8]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_label, y_label, test_size=0.25, random_state=42)

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
clf = RandomForestClassifier(n_estimators=100)
model = clf.fit(x_train, y_train)
pred = model.predict(x_test)
print('Accuracy of ', accuracy_score(pred, y_test)*100)

Accuracy of  95.0


In [10]:
test_data=pd.read_csv('./data/tests.csv')
test_data.head(5)

,First_pokemon,Second_pokemon
0,129,117
1,660,211
2,706,115
3,195,618
4,27,656


In [11]:
new_test_data=test_data[["First_pokemon","Second_pokemon"]].replace(pokemon.Name)
new_test_data.head()

,First_pokemon,Second_pokemon
0,Seaking,Lickitung
1,Ferrothorn,Quagsire
2,Thundurus Therian Forme,Hitmonlee
3,Flaaffy,Maractus
4,Spearow,Alomomola


In [12]:
final_data=normalization(test_data)
pred=model.predict(final_data)
test_data["Winner"]=[test_data["First_pokemon"][i] if pred[i]==0 else test_data["Second_pokemon"][i] for i in range(len(pred))]

In [16]:
cols = ["First_pokemon","Second_pokemon","Winner"]  
combats_name = test_data[cols].replace(pokemon.Name)
combats_name[63:64]

,First_pokemon,Second_pokemon,Winner
63,Mr. Mime,Psyduck,Mr. Mime


In [14]:
def batalhaPokemon(Primeiro, Segundo):
    data_df = pd.DataFrame.from_dict({0:[Primeiro, Segundo]},\
                                     columns=['First_pokemon', 'Second_pokemon'],\
                                     orient='index')
    stats=["Hit Points","Attack","Defense","Sp. Atk","Sp. Def","Speed","Legendary"]
    stats_df=pokemon[stats].T.to_dict("list")
    one=data_df.First_pokemon.map(stats_df)
    two=data_df.Second_pokemon.map(stats_df)
    temp_list=[]
    for i in range(len(one)):
        temp_list.append(np.array(one[i])-np.array(two[i]))
    new_test = pd.DataFrame(temp_list, columns=stats)
    resultado = model.predict(new_test)
    if resultado[0] == 0:
        campeao = data_df[["First_pokemon"]].replace(pokemon.Name)
    else :
        campeao = data_df[["Second_pokemon"]].replace(pokemon.Name) 
    cols = ["First_pokemon","Second_pokemon","Winner"]    
    data_df['Winner'] = [data_df["First_pokemon"][i] if pred[i]==0 else data_df["Second_pokemon"][i] for i in range(len(resultado))]
    data_df = data_df[cols].replace(pokemon.Name)
    return data_df

In [15]:
batalhaPokemon(132,60)

,First_pokemon,Second_pokemon,Winner
0,Mr. Mime,Psyduck,Mr. Mime
